In [106]:
#imports
import requests
import pandas as pd
import psycopg2
import warnings
warnings.filterwarnings('ignore')

In [104]:
#Set up parameters and URL for game request. Enter a game ID
GAME_ID = '2019020940'

URL = "http://statsapi.web.nhl.com/api/v1/game/%s/feed/live" % GAME_ID

TEST_QUERY = '''
SELECT ordinalNum AS period, periodTime, event, playerFullName, description
FROM game_data
WHERE eventTypeId = 'GOAL'
ORDER BY dateTime ASC;
'''

In [17]:
#EXTRACT
#submit request to NHL REST API and collect JSON game data
jsonData = requests.get(url = URL).json()

In [77]:
#TRANSFORM part 1
#separate and flatten JSON data into a flat CSV for DB consumption

data = []

for play in jsonData['liveData']['plays']['allPlays']:

    #lets unpack the desired data from the JSON/Dict into a flat dict to load into a pandas DataFrame     
    
    #check to see if there is player data for the play i.e. a scoring play, faceoff, hit, etc
    #for the sake of brevity, we are only collecting the first player listed in the event (index = 0)
    #a more robust implementation is required for handling N number of players (unassisted goal vs N assisting players)
    if 'players' in play:
    
        data.append({   'gameId'               : GAME_ID,
                        'event'                : play['result']['event'],
                        'eventCode'            : play['result']['eventCode'],
                        'eventTypeId'          : play['result']['eventTypeId'],
                        'description'          : play['result']['description'],
                        'eventId'              : play['about']['eventId'],
                        'playerId'             : play['players'][0]['player']['id'],
                        'playerFullName'       : play['players'][0]['player']['fullName'],
                        'period'               : play['about']['period'],
                        'periodType'           : play['about']['periodType'],
                        'ordinalNum'           : play['about']['ordinalNum'],
                        'periodTime'           : play['about']['periodTime'],
                        'periodTimeRemaining'  : play['about']['periodTimeRemaining'],
                        'dateTime'             : play['about']['dateTime']                

                     })
        
    #if there is no player data, simply set player info to Null or None
    else:
        data.append({   'gameId'               : GAME_ID,
                        'event'                : play['result']['event'],
                        'eventCode'            : play['result']['eventCode'],
                        'eventTypeId'          : play['result']['eventTypeId'],
                        'description'          : play['result']['description'],
                        'eventId'              : play['about']['eventId'],
                        'playerId'             : None,
                        'playerFullName'       : None,
                        'period'               : play['about']['period'],
                        'periodType'           : play['about']['periodType'],
                        'ordinalNum'           : play['about']['ordinalNum'],
                        'periodTime'           : play['about']['periodTime'],
                        'periodTimeRemaining'  : play['about']['periodTimeRemaining'],
                        'dateTime'             : play['about']['dateTime']                

                 })

#load aggregated data into DataFrame
df = pd.DataFrame(data)

In [81]:
#TRANSFORM part 2

#enforce column ordering
columnOrderList = ['gameId', 'eventId', 'event', 'eventCode', 'eventTypeId', 'description', 'playerId',
                   'playerFullName', 'period', 'periodType', 'ordinalNum', 'periodTime', 'periodTimeRemaining', 'dateTime']
df = df[columnOrderList]



#write DataFrame to CSV file
#using pipe character to separate since there are commas in the description field that can break import to DB
#there are better ways to handle this, but using this workaround in the sake of brevity
df.to_csv('csvDump.csv', index=False, sep = '|', header=False)


In [93]:
#LOAD

#prerequisite: run the associated table_init.sql to create and configure the table to store the data

#set up connection to DB
connectionObj = psycopg2.connect(dbname = 'XXX',
                                 host = 'XXX',
                                 port = 'XXX', 
                                 user = 'XXX', 
                                 password = 'XXX')

connectionObj.autocommit=True

cursorObj = connectionObj.cursor()

#load data into DB from CSV data file
with open('csvDump.csv', 'r') as csvData:
    cursorObj.copy_from(csvData, 'game_data', sep='|', null='')


In [105]:
#TEST

#Let's query the newly loaded data for goal scoring events to verify it has loaded correctly
cursorObj.execute(TEST_QUERY)

test_df = pd.DataFrame(cursorObj.fetchall(), columns = [desc[0] for desc in cursorObj.description])

test_df

,period,periodtime,event,playerfullname,description
0,1st,01:26,Goal,Gabriel Vilardi,"Gabriel Vilardi (1) Snap Shot, assists: Kurtis..."
1,1st,07:03,Goal,Aleksander Barkov,"Aleksander Barkov (19) Wrist Shot, assists: Jo..."
2,1st,08:43,Goal,Aleksander Barkov,"Aleksander Barkov (20) Snap Shot, assists: Jon..."
3,1st,15:12,Goal,Drew Doughty,"Drew Doughty (7) Slap Shot, assists: Adrian Ke..."
4,2nd,03:19,Goal,Jonathan Huberdeau,"Jonathan Huberdeau (22) Snap Shot, assists: Ev..."
5,2nd,10:18,Goal,Trevor Moore,"Trevor Moore (5) Tip-In, assists: Drew Doughty..."
6,2nd,18:42,Goal,Martin Frk,"Martin Frk (5) Snap Shot, assists: Gabriel Vil..."
7,3rd,00:36,Goal,Mike Hoffman,"Mike Hoffman (23) Slap Shot, assists: Jonathan..."
8,3rd,12:45,Goal,Ben Hutton,"Ben Hutton (4) Wrist Shot, assists: Alex Iafal..."


SUCCESS!